In [16]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [17]:
# 📌 Step 3: Load the Gemini API Key
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found in .env file.")

os.environ["GOOGLE_API_KEY"] = api_key
print("✅ Gemini API key loaded")


✅ Gemini API key loaded


In [18]:
# 📌 Step 4: Load and Chunk the PDF
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    
    chunks = splitter.split_documents(pages)
    return chunks

# Change to your actual file path
pdf_path = "RAG_Agent.pdf"
documents = load_pdf(pdf_path)
print(f"✅ Loaded and split into {len(documents)} chunks.")


✅ Loaded and split into 8 chunks.


In [19]:
# 📌 Step 5: Create Embeddings and Vector Store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_db = FAISS.from_documents(documents, embeddings)
print("✅ FAISS vector store created.")


✅ FAISS vector store created.


In [23]:
# Define the custom prompt first
custom_prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
You are a helpful assistant that only answers based on the provided context.
Do not use prior knowledge.

Context:
{context}

Question:
{query}

Only answer from the context. If the answer is not in the context, say "I don't know."
"""
)

# Build the QA chain with Gemini
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",  # ✅ Full model path
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=40
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    return_source_documents=True,
    chain_type="stuff",
    chain_type_kwargs={"prompt": custom_prompt}
)

print("✅ QA chain ready.")


✅ QA chain ready.


In [ ]:
# # 📌 Step 6: Build the RetrievalQA Chain
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-flash",
#     # google_api_key=GOOGLE_API_KEY,
#     temperature=0.3,
#     max_output_tokens=1024,
#     top_p=0.8,
#     top_k=40
# )

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=vector_db.as_retriever(),
#     return_source_documents=True,
#     chain_type="stuff",
#     chain_type_kwargs={"prompt": custom_prompt}
# )
# print("✅ QA chain ready.")


✅ QA chain ready.


In [ ]:
# 📌 Step 7: Ask a Question
question = "I am near the belpada station and want to deliver the food to bharati vidyapeeth boys hostel"

response = qa_chain({"question": question})
print("🧠 Answer:", response["result"])

# Show relevant source context
for i, doc in enumerate(response["source_documents"]):
    print(f"\n📄 Source {i+1}:\n{doc.page_content}")


ValueError: Missing some input keys: {'query'}

In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

models = genai.list_models()
for model in models:
    print(model.name, "→", model.supported_generation_methods)


c:\Users\hp\OneDrive\Desktop\Ai_Agent\agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


models/embedding-gecko-001 → ['embedText', 'countTextTokens']
models/gemini-1.5-pro-latest → ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 → ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro → ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest → ['generateContent', 'countTokens']
models/gemini-1.5-flash → ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 → ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest → ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-flash-preview-05-20 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateConten